In [42]:
import pandas as pd 
import numpy as np
import requests
import xlsxwriter
import math

In [43]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [44]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [45]:
from secretsAPI import IEX_CLOUD_API_TOKEN

In [46]:
symbol = 'AAPL'
api_url =f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token=sk_86b27e27ee6b4208b6f90e6edbc16d18'
print(api_url)
data = requests.get(api_url).json()
print(data)

https://api.iex.cloud/v1/data/core/quote/AAPL?token=sk_86b27e27ee6b4208b6f90e6edbc16d18
[{'avgTotalVolume': 61623486, 'calculationPrice': 'close', 'change': 1.22, 'changePercent': 0.00724, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 168.78, 'iexAskSize': 220, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 168.99, 'iexCloseTime': 1712752959695, 'iexLastUpdated': 1712752959695, 'iexMarketPercent': 0.0007256321196959024, 'iexOpen': 168.99, 'iexOpenTime': 1712752959695, 'iexRealtimePrice': 168.99, 'iexRealtimeSize': 50, 'iexVolume': 478, 'lastTradeTime': 1712692799907, 'latestPrice': 169.67, 'latestSource': 'Close', 'latestTime': 'April 9, 2024', 'latestUpdate': 1712692800561, 'latestVolume': None, 'low': None,

In [47]:
price = data[0]['latestPrice']
marketcap = data[0]['marketCap']/1000000000000
print(marketcap)

2.62002394927


In [48]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
df = pd.DataFrame(columns = my_columns)
df

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [49]:
df = df.append(pd.Series(['AAPL', data[0]['latestPrice'], data[0]['marketCap'], 'N/A'], index = my_columns), ignore_index = True)
df

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,169.67,2620023949270,N/A


In [50]:
for symbol in stocks["Ticker"][:5]:
    api_url =f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token=sk_86b27e27ee6b4208b6f90e6edbc16d18'
    data = requests.get(api_url).json()
    df = df.append(pd.Series([symbol, data[0]['latestPrice'], data[0]['marketCap'], 'N/A'], index = my_columns), ignore_index = True)


In [51]:
df

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,169.67,2620023949270,N/A
1,A,147.40,43196348862,N/A
2,AAL,13.95,9133857583,N/A
3,AAP,78.46,4676959095,N/A
4,AAPL,169.67,2620023949270,N/A
5,ABBV,170.14,301257877688,N/A


In [52]:
# So the previous task was very time consuming so we will divide it into batches/ chunks 

def chunks(lst , n):
    for i in range(0,len(lst), n):
        yield lst[i:i+n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/stock/market/batch?symbols={symbol_string}&types=quote&token=sk_86b27e27ee6b4208b6f90e6edbc16d18'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        if symbol not in data or 'quote' not in data[symbol]:
            print(f'No data for {symbol}')
            continue
            
        latest_price = data[symbol]['quote'].get('latestPrice', 'N/A')
        market_cap = data[symbol]['quote'].get('marketCap', 'N/A')
        
        final_df = final_df.append(pd.Series([symbol, latest_price, market_cap, 'N/A'], index=my_columns), ignore_index=True)

final_df

No data for ABC
No data for ANTM
No data for DISCA
No data for FBHS
No data for HFC
No data for NLOK
No data for RE
No data for VIAC
No data for WLTW


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,147.40,43196348862,N/A
1,AAL,13.95,9133857583,N/A
2,AAP,78.46,4676959095,N/A
3,AAPL,169.67,2620023949270,N/A
4,ABBV,170.14,301257877688,N/A
...,...,...,...,...
491,YUM,137.02,38571130000,N/A
492,ZBH,129.40,26586711759,N/A
493,ZBRA,306.17,15736382066,N/A
494,ZION,43.51,6422033360,N/A


In [14]:
# Calculating the nuber shares to buy

In [19]:
portfolio_size = input ("Enter the value of your portfolio : ")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number \n Try again")
    portfolio_size = input('Enter the value of portfolio : ')
    val = float(portfolio_size)

Enter the value of your portfolio : 1000000


In [20]:
# What we have to do is to divide our money equally into every stock 
# And see what value of money will we invest in every company stock

# This is called position_size

position_size= val/len(final_df.index)
position_size

2016.1290322580646

In [23]:
# Now we have to find out how many shares of each stock we have to purchase 

for i in range(0,len(final_df.index)):
    final_df.loc[i,'Number Of Shares to Buy'] = math.floor(position_size/final_df.loc[i,'Price']) 

In [24]:
 final_df

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,147.40,43196348862,13
1,AAL,13.95,9133857583,144
2,AAP,78.46,4676959095,25
3,AAPL,169.67,2620023949270,11
4,ABBV,170.14,301257877688,11
...,...,...,...,...
491,YUM,137.02,38571130000,14
492,ZBH,129.40,26586711759,15
493,ZBRA,306.17,15736382066,6
494,ZION,43.51,6422033360,46


In [36]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Recommended Trades', index = False)


In [37]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [38]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


In [39]:
writer.save()